In [11]:
import torch
import os
import sys
import numpy as np
import onnx
import onnxruntime
import time
sys.path.append(os.path.abspath('..'))

In [2]:
from model.net import ConvTasNet
net = ConvTasNet(N=512, L=40, B=128,
                 Sc=128, H=512, P=3,
                 X=8, R=3, C=2, N_ED=2,
                 mask_act='relu', attention=[1, 1])

In [6]:
dummy_input = torch.randn(1,1,32000)
torch.onnx.export(net,
                  dummy_input,
                  'model.onnx',
                  input_names=['input'],
                  output_names=['output'],
                  dynamic_axes={'input': {2: 'utt'},
                                'output': {2: 'utt'}},
                  opset_version=11,
                  )

In [9]:
model = onnx.load('model.onnx')
onnx.checker.check_model(model)

In [10]:
model = onnxruntime.InferenceSession('model.onnx')

In [22]:
wav = np.empty((1,1,16000*4),dtype=np.float32)
input = {model.get_inputs()[0].name: wav}
tic = time.time()
out = model.run(None, input)[0]
print(time.time()-tic)

0.9369978904724121


In [23]:
with torch.no_grad():
    tic = time.time()
    out_torch = net(torch.tensor(wav))
    print(time.time()-tic)

1.751145839691162
